In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta
%matplotlib qt

## Definições

In [38]:
def crop_roi(img):
    return img[150:173, 341:350]

window_width = 5
reference = 1
limiar = 90   

In [ ]:
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

## Capturar um frame

In [31]:
cap = cv2.VideoCapture('video.AVI')
ret, frame = cap.read()
cv2.imwrite("frame.png", frame)
# cv2.imwrite("frame.png", crop_roi(frame))

True

## Calcular

In [53]:
cap = cv2.VideoCapture('video.AVI')
fps = cap.get(cv2.CAP_PROP_FPS)
window_width = 50
reference = 102

means = []
while True:
    ret, frame = cap.read()
    if frame is None:
        break
    crop = crop_roi(frame)
    grey = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    mean = np.mean(grey)
    means.append(np.mean(grey))
df = pd.DataFrame({'mean': means})
df['n'] = df.index.astype(int)
df['second'] = df['n'].apply(lambda x: x/fps)
df['time'] = df['second'].apply(lambda x: timedelta(seconds=x))

## Plotar gráfico das médias

In [54]:
df.plot(x='second', y='mean')

## Gerar vídeo

In [49]:
def write_timestamp(frame, row):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,500)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2

    cv2.putText(frame,str(row['time']), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

df['include'] = df['mean'].apply(lambda x: True if x < limiar else False)
image_array = []
for i, row in df[df.include].iterrows():
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    write_timestamp(frame, row)
    image_array.append(frame)
height, width, layers = image_array[0].shape
size = (width,height)
out = cv2.VideoWriter(f"files\\video.avi",cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(image_array)):
    out.write(image_array[i])
out.release()